In [120]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [136]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
webpage=requests.get('https://www.foreclosureindia.com/bank-auctions/amssam/1',headers=headers).text

In [137]:
soup=BeautifulSoup(webpage,'lxml')

In [138]:
Auction=soup.find_all('tr',class_='top aligned')

In [139]:
len(Auction)

10

In [140]:
Auction_ID=soup.find('h4',class_='ui center aligned header').text
print(Auction_ID)

1291371


In [141]:
Bank=soup.find_all('td')[1].text.strip()
print(Bank)

Tata Capital Housing Finance ltd.


In [142]:
Property_Details=soup.find_all('td')[2].text.strip()
print(Property_Details)

Land and Building


In [143]:
Application_Deadline=soup.find_all('td')[3].text.strip()
print(Application_Deadline)

Kochi


In [144]:
Auction_Date=soup.find_all('td')[4].text.strip()
print(Auction_Date)

29-09-2023


In [145]:
Reserve_Price=soup.find_all('td')[5].text.strip()
print(Reserve_Price)

7500000


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup


Auction_ID = []
Bank = []
Property_Details = []
Application_Deadline = []
Auction_Date = []
Reserve_Price = []


for row in Auction:
    columns = row.find_all('td')
    if len(columns) >= 6:
        Auction_ID.append(columns[0].find('h4', class_='ui center aligned header').text)
        Bank.append(columns[1].text.strip())
        Property_Details.append(columns[2].text.strip())
        Application_Deadline.append(columns[3].text.strip())
        Auction_Date.append(columns[4].text.strip())
        Reserve_Price.append(columns[5].text.strip())


df = pd.DataFrame({
    'Auction_ID': Auction_ID,
    'Bank': Bank,
    'Property_Details': Property_Details,
    'Application_Deadline': Application_Deadline,
    'Auction_Date': Auction_Date,
    'Reserve_Price': Reserve_Price
})


df.index = df.index + 1


In [147]:
df

,Auction_ID,Bank,Property_Details,Application_Deadline,Auction_Date,Reserve_Price
1,1291371,Tata Capital Housing Finance ltd.,Land and Building,Kochi,29-09-2023,7500000
2,1291237,Ashoknagar Co-operative bank Ltd,Residential Plot,Bengaluru,25-09-2023,14000000
3,1291223,Tata Capital Housing Finance ltd.,Land,Guntur,29-09-2023,1450000
4,1291222,Tata Capital Housing Finance ltd.,Residential Plot,Guntur,29-09-2023,799000
5,1291221,Tata Capital Housing Finance ltd.,Plot,Vijayawada,29-09-2023,575000
6,1291220,Tata Capital Housing Finance ltd.,Plot,Vijayawada,29-09-2023,1460000
7,1291219,Tata Capital Housing Finance ltd.,Residential Plot,Vijayawada,29-09-2023,590000
8,1291218,Tata Capital Housing Finance ltd.,Land and Building,Vijayawada,29-09-2023,1500000
9,1291217,Tata Capital Housing Finance ltd.,Residential Flat,Vijayawada,29-09-2023,1260000
10,1291216,Tata Capital Housing Finance ltd.,House,Rajahmundry,29-09-2023,1500000


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
import time


def scrape_page(page_number):
    url = f'https://foreclosureindia.com/bank-auctions/assam/{page_number}'  # Replace with the actual URL
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        Auction_ID = []
        Bank = []
        Property_Details = []
        Application_Deadline = []
        Auction_Date = []
        Reserve_Price = []

       
        for row in soup.find_all('tr', class_='top aligned'):
            columns = row.find_all('td')
            if len(columns) >= 6:
                Auction_ID.append(columns[0].find('h4', class_='ui center aligned header').text)
                Bank.append(columns[1].text.strip())
                Property_Details.append(columns[2].text.strip())
                Application_Deadline.append(columns[3].text.strip())
                Auction_Date.append(columns[4].text.strip())
                Reserve_Price.append(columns[5].text.strip())

        
        return {
            'Auction_ID': Auction_ID,
            'Bank': Bank,
            'Property_Details': Property_Details,
            'Application_Deadline': Application_Deadline,
            'Auction_Date': Auction_Date,
            'Reserve_Price': Reserve_Price
        }
    else:
        print(f"Failed to fetch page {page_number}. Status code: {response.status_code}")
        return None


start_page = 1
end_page = 2000  


page_numbers = list(range(start_page, end_page + 1))


all_data = []


def process_pages(page_number):
    data = scrape_page(page_number)
    if data:
        all_data.append(data)


with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(process_pages, page_numbers)


dfs = [pd.DataFrame(data) for data in all_data]
df = pd.concat(dfs, ignore_index=True)



time.sleep(1)



In [149]:
df

,Auction_ID,Bank,Property_Details,Application_Deadline,Auction_Date,Reserve_Price
0,1291230,Punjab National Bank,Commercial Property in Kamrup,11-09-2023,12-09-2023,3800000
1,1291227,Punjab National Bank,Flat in Kamrup,11-09-2023,12-09-2023,3000000
2,1291214,Punjab National Bank,Flat in Dispur,11-09-2023,12-09-2023,5860000
3,1291213,Punjab National Bank,Land in Golaghat,11-09-2023,12-09-2023,7550000
4,1291188,Punjab National Bank,Land and Building in Sibsagar,20-09-2023,21-09-2023,850000
...,...,...,...,...,...,...
668,427679,Punjab National Bank,Plot in Assam,20-03-2019,21-03-2019,3500000
669,427680,Punjab National Bank,Plot in Assam,20-03-2019,21-03-2019,495000
670,423893,Canara Bank,Land in Assam,22-03-2019,25-03-2019,2011000
671,423894,Canara Bank,Land and Building in Assam,22-03-2019,25-03-2019,2800000


In [150]:
csv_file = "foreclosure_india_data.csv"
df.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8')

print(f"Data has been appended to {csv_file}")


Data has been appended to foreclosure_india_data.csv
